In [1]:
import pandas as pd
import re
import json
import gzip
import shutil
import gdown
import pyarrow as pa
import ast
import numpy as np

In [2]:
def compare_date_columns(df):
    # Ensure 'reviewTime' column is in datetime format
    df['reviewTime'] = pd.to_datetime(df['reviewTime'], format='%m %d, %Y')

    # Compare 'unixReviewTime' and 'reviewTime' for equality
    df['same_date'] = df['reviewTime'].dt.date == pd.to_datetime(df['unixReviewTime'], unit='s').dt.date

    return df

In [4]:
import pandas as pd
import numpy as np

def identificar_duplicados_metas(df):
    # Crear una copia del DataFrame
    df_copy = df.copy()

    # Convertir las columnas de diccionarios a cadenas de texto
    for col in df_copy.columns:
        df_copy[col] = df_copy[col].apply(lambda x: str(x) if isinstance(x, dict) else x)

    # Seleccionar solo las columnas que no contienen objetos de tipo 'dict'
    columnas_no_dict = [col for col in df_copy.columns if not df_copy[col].apply(lambda x: isinstance(x, dict)).any()]

    # Buscar duplicados en las columnas seleccionadas
    duplicados = df_copy[df_copy.duplicated(subset=columnas_no_dict, keep=False)]
    return duplicados



In [5]:


def identificar_duplicados(df):
    # Filtrar las columnas que no son de tipo numpy.ndarray
    columnas_validas = [col for col in df.columns if not isinstance(df[col].iloc[0], np.ndarray)]
    
    # Crear un nuevo DataFrame solo con las columnas válidas
    df_valido = df[columnas_validas]
    
    # Buscar duplicados en el DataFrame válido
    duplicados = df_valido[df_valido.duplicated(keep=False)]
    return duplicados

def identificar_nulos(df):
    nulos = df[df.isnull().any(axis=1)]
    return nulos


In [6]:
import pandas as pd

def convertir_minusculas(df):
    # Crear una copia del DataFrame
    df_copy = df.copy()

    # Aplicar str.lower() a todas las columnas de tipo 'object'
    df_copy = df_copy.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    return df_copy


#ALL_BEAUTY

In [7]:
all_beauty=pd.read_parquet('reviews_All_Beauty.parquet')
meta_beauty=pd.read_parquet('meta_All_Beauty.parquet')

Vamos a identificar nulos y duplicados de los datasets

In [8]:
duplicados=identificar_duplicados(all_beauty)
print(duplicados)

Empty DataFrame
Columns: [reviewerID, asin, reviewerName, reviewText, overall, summary, unixReviewTime, reviewTime]
Index: []


In [9]:
nulos=identificar_nulos(all_beauty)
print(nulos)

           reviewerID        asin reviewerName helpful  \
959     APUGL98BRKRA7  B00014EB6O         None  [0, 0]   
960     AM29DOTJ3930J  B00014EB6O         None  [0, 0]   
1487   A3GAO75P39A7C1  B0001ZA4EQ         None  [0, 0]   
1551   A2WXCBXSBAZO3Y  B00020NWQ2         None  [0, 0]   
2447   A19BTOF425WVIR  B000624Z6C         None  [0, 0]   
...               ...         ...          ...     ...   
30168  A2KBNKNP9XC4LQ  B008OBGY98         None  [0, 0]   
30321   A771KH2WY8QRG  B008X3658W         None  [0, 0]   
30683  A3N3E1CEXX6VPE  B009PQB24S         None  [0, 0]   
31249  A1ZTFVT4271GWF  B00APDIFN6         None  [0, 0]   
33021  A2WA58FWWWBP82  B00E1UZ640         None  [0, 0]   

                                              reviewText  overall  \
959    Another item that help me post surgery. I used...      5.0   
960    The only aloe vera juice I will drink.  It tas...      5.0   
1487                                       Awesome stuff      5.0   
1551   Worked good just a l

In [10]:
all_beauty_final = convertir_minusculas(all_beauty)

In [11]:
all_beauty_final

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,a1zznihf0ty3p5,0977217213,josh,"[0, 0]",i like but they are not stickers so they are a...,5.0,great,1397865600,"04 19, 2014"
1,a3alvbkmdjnwq9,1030070148,beauman,"[0, 0]","i mistakenly ordered two sets, i asked if i co...",5.0,colored contat lens,1287187200,"10 16, 2010"
2,a1dxtzs7gtw0na,189112966x,"sandra l. good ""goodgal""","[7, 7]",what a handy little item. it's chock full of ...,5.0,splendid!!,1346544000,"09 2, 2012"
3,a1tyq9p2erhf9v,8950947714,judy,"[0, 0]",i just like the smell of this shampoo which is...,5.0,i love the smell of this shampoo,1375920000,"08 8, 2013"
4,a2l3xc0aivdpka,8985802089,zong weiwei,"[0, 0]",i've been using l'oreal products and tried sev...,5.0,shiseido aquair is good for freezy hair.,1365379200,"04 8, 2013"
...,...,...,...,...,...,...,...,...,...
34620,a241ds00ohez1h,b00kpfrv0m,anna b.,"[0, 0]",only been using it for about a week can't see ...,4.0,four stars,1405555200,"07 17, 2014"
34621,ao3iovo1d3d8g,b00kqrl6au,dawn,"[0, 0]",this scrub leaves you feeling renewed from sho...,5.0,amazing!!,1402444800,"06 11, 2014"
34622,a2cvj5c716ckvg,b00krxcbi4,karen galvez,"[0, 0]",wow! gorgeous tie! price is awesome and went p...,5.0,gorgeous tie,1405209600,"07 13, 2014"
34623,a2w389f34ec744,b00kvm9aq2,rasheedah johnson,"[0, 0]","it took a while to find a good setting, but i ...",4.0,better than average. but the ring fits ..,1403654400,"06 25, 2014"


In [12]:
all_beauty.drop(columns=['helpful', 'reviewerName', 'reviewTime'], inplace=True)

Se encontraron Nulos pero estaban en la columna que se iban a eliminar por no ser tomadas en el analisis final se convierte todo a minuscula para podes trabara y sea mas facil la validacion de datos 

In [13]:
meta_beauty

,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,0977217213,"{'also_bought': ['0977217205', 'B000VUYS0W', '...",Water Blessing Labels Prosperity Collection 16 pc,9.94,"{'Beauty': 234894.0, 'Clothing': None, 'Grocer...",http://ecx.images-amazon.com/images/I/31ul4drz...,Ma&#39;s India,[[All Beauty]],The Spirit of Water 16 REUSABLE STATIC CLING W...
1,1030070148,None,Colored Cosmetic Lens in Twilight Blue,NaN,"{'Beauty': 1136984.0, 'Clothing': None, 'Groce...",http://ecx.images-amazon.com/images/I/11ZF32FL...,None,[[All Beauty]],Comfortable and FUN to wear.\nRadiant color th...
2,189112966X,"{'also_bought': ['1891129732', 'B006GT1MDI', '...",Sacred Wisdom Chart: Tarot Divination,5.99,"{'Beauty': 95030.0, 'Clothing': None, 'Grocery...",http://ecx.images-amazon.com/images/I/51BSkLxS...,Helion Communications,[[All Beauty]],HELION TAROT DIVINATION REFERENCE CHART The im...
3,8950947714,None,Shiseido Aquair Shampoo + Conditioner + Treatm...,NaN,"{'Beauty': 524492.0, 'Clothing': None, 'Grocer...",http://ecx.images-amazon.com/images/I/41EXaaVN...,None,[[All Beauty]],None
4,8985802089,None,Shiseido Aquair Shampoo/Conditioner/Spray SET ...,NaN,"{'Beauty': 878035.0, 'Clothing': None, 'Grocer...",http://ecx.images-amazon.com/images/I/41BYNfTV...,None,[[All Beauty]],None
...,...,...,...,...,...,...,...,...,...
6575,B00KQRL6AU,None,Melissa's Virgin Grapefruit Scrub,NaN,None,http://ecx.images-amazon.com/images/I/41AgX5tw...,None,[[All Beauty]],"A natural pairing, Organic Extra Virgin Coconu..."
6576,B00KRR15UA,"{'also_bought': None, 'also_viewed': ['B00IA8L...",Maybelline the Provocative Look the Colossal M...,17.99,"{'Beauty': 401194.0, 'Clothing': None, 'Grocer...",http://ecx.images-amazon.com/images/I/51xMCtxr...,None,[[All Beauty]],Maybelline the Provocative Look the Colossal M...
6577,B00KRXCBI4,"{'also_bought': None, 'also_viewed': ['B00KRXC...",Dark Red Burgundy Gradient Swirl Paisley Men's...,6.99,"{'Beauty': 521208.0, 'Clothing': None, 'Grocer...",http://ecx.images-amazon.com/images/I/51zM1byB...,onestopmall,[[All Beauty]],Item description: This kind of tie would be pe...
6578,B00KVM9AQ2,"{'also_bought': None, 'also_viewed': ['B00L9G2...",Bedroom Kandi Rise &amp; Shine Vibrating Cock ...,NaN,"{'Beauty': 121818.0, 'Clothing': None, 'Grocer...",http://ecx.images-amazon.com/images/I/41YHO-oK...,None,[[All Beauty]],A blissfully couple friendly vibe from OhMiBod...


In [14]:
meta_beauty.drop(columns=['imUrl', 'related', 'salesRank'], inplace= True)

Se eliminan las columnas que no se van a utilizar

In [15]:
nulos=identificar_nulos(meta_beauty)
print(nulos)

            asin                                              title  price  \
1     1030070148             Colored Cosmetic Lens in Twilight Blue    NaN   
3     8950947714  Shiseido Aquair Shampoo + Conditioner + Treatm...    NaN   
4     8985802089  Shiseido Aquair Shampoo/Conditioner/Spray SET ...    NaN   
5     9602400048  Amino Genesis Cocoon Rejuvenating Body Cr&egra...   9.00   
6     9744914572  Finale Whitening Cream 30g. Armpit/inner thigh...    NaN   
...          ...                                                ...    ...   
6574  B00KPFRV0M                                   Neckline Slimmer    NaN   
6575  B00KQRL6AU                  Melissa's Virgin Grapefruit Scrub    NaN   
6576  B00KRR15UA  Maybelline the Provocative Look the Colossal M...  17.99   
6578  B00KVM9AQ2  Bedroom Kandi Rise &amp; Shine Vibrating Cock ...    NaN   
6579  B00KY502J0  Disney Frozen Kid Children Button Hair Clips (...   8.99   

     brand      categories                                     

Se encuentran nulos en precio pero no se eliminan para vere el tratamiento que se le puede hacer 

In [16]:
meta_beauty.to_parquet('meta_beauty2.parquet',index=False)
all_beauty_final.to_parquet('all_beauty2.parquet',index=False)

#AMAZON_FASHION

In [17]:
fashion=pd.read_parquet('reviews_Amazon_Fashion.parquet')
meta_Amazon_Fashion=pd.read_parquet('meta_Amazon_Fashion.parquet')

In [18]:
duplicados=identificar_duplicados(fashion)
print(duplicados)

Empty DataFrame
Columns: [reviewerID, asin, reviewerName, reviewText, overall, summary, unixReviewTime, reviewTime]
Index: []


In [19]:
nulos=identificar_nulos(fashion)
print(nulos)

           reviewerID        asin reviewerName helpful  \
1040   A288CB718EI3AQ  B0000YRQEE         None  [0, 0]   
1041   A3H53O51L8JXOT  B0000YRQEE         None  [0, 0]   
1222   A290PFPBHZ960A  B0001CULSS         None  [0, 0]   
2352   A20TID11IU40JF  B0006LTEPK         None  [0, 0]   
2353   A3F6LNDW9470HA  B0006LTEPK         None  [0, 0]   
...               ...         ...          ...     ...   
77096  A29PW61SSIDUJR  B00DSXL84S         None  [0, 0]   
77189  A19FMYSLGZG57I  B00DV0NPN0         None  [0, 0]   
77947  A2TLZDK4U00I1S  B00E6KBPCM         None  [0, 0]   
78913   AGH5TDYLOY34X  B00F27CYP0         None  [0, 0]   
79274  A2RD4166F1W0NV  B00FGFJ4WE         None  [0, 0]   

                                              reviewText  overall  \
1040   The listing said two garment bags were include...      3.0   
1041   Worked well to protect garments that were then...      4.0   
1222   THE WATCH KNIFE WAS HIGH QUALITY AND WHAT I EX...      5.0   
2352   Great fit. Great pri

In [20]:
fashion.drop(columns=['helpful', 'reviewerName', 'reviewTime'], inplace=True)

In [21]:
fashion2 = convertir_minusculas(fashion)

Se eliminan columnas que no se utilizaran donde estan las columnas con muchos datos nulos se revisa si hay duplicados no se encontraron y se pasa todo el texto a minuscula para ser mas sencillo trabajarlo

In [22]:
fashion2

,reviewerID,asin,reviewText,overall,summary,unixReviewTime
0,a2lotv0twu6uqi,1880241196,i ordered this for christmas for my grandson a...,1.0,did not read the whole paragragh,1355270400
1,a1h1l1mzxlmgj3,3890968740,i got this watch for my 13 year old daughter. ...,4.0,so far still ticking away,1362182400
2,a2h3f19k13qqox,5555014597,"3rd year running i have brought these, takes a...",5.0,"3rd year purchased, always good hats!",1357257600
3,a2cpkwdhow37et,5555014597,"quality construction, good looking hat. given ...",5.0,"nice construction, good looking.",1392681600
4,a7puhk9fik5o7,5555014597,"i love it! i have followed the show for years,...",5.0,top gear baseball cap review,1397433600
...,...,...,...,...,...,...
83492,a1s96ym6gkbgrp,b00lf6l46w,fits well and is light.,5.0,nice hat,1406073600
83493,a35lv1hl059g9k,b00lgtrmpu,it arrived faster than i expected considering ...,4.0,it's thick but comfortable.,1405036800
83494,axvc9zqcqqt7k,b00lgxw620,i love the quality of this belt! very nice lea...,5.0,quality is the best!,1405987200
83495,a36tm2ivfj7tpp,b00lgxw620,finally a belt that came all in one usage! air...,5.0,all in one belt,1404259200


In [ ]:
meta_Amazon_Fashion.drop(columns=['imUrl', 'related', 'salesRank','brand'], inplace= True)

In [ ]:
meta_Amazon_Fashion

,asin,title,price,categories,brand,description
0,1880241196,Bird Watching For Kids - A Special Publication...,4.50,[[Amazon Fashion]],None,None
1,3890968740,NEW Mango Yellow Mix Colorful Jelly Silicone S...,NaN,[[Amazon Fashion]],None,None
2,5555014597,Top Gear Official Merchandise - Top Gear Baseb...,19.50,[[Amazon Fashion]],None,None
3,7842955883,Blessed By Pope Benedict XVI St Benedict One-d...,9.99,[[Amazon Fashion]],None,None
4,8037200221,Envirosax Greengrocer Series Bag 7 Guava,NaN,[[Amazon Fashion]],None,None
...,...,...,...,...,...,...
24140,B00LDSPA5S,Flora Print Vintage High Waisted Bikini Sets S...,NaN,[[Amazon Fashion]],None,None
24141,B00LF6L46W,Nike Golf Tech Swoosh Cap,NaN,[[Amazon Fashion]],None,None
24142,B00LGTRMPU,Cottory 2 Colors Mens Cotton Loose Harem Pants...,25.98,[[Amazon Fashion]],Cottory,None
24143,B00LGXW620,"Metal free belt/non metal, airport friendly be...",NaN,[[Amazon Fashion]],None,None


Se eliminan columnas donde hay informacion que no se utilizara y su mayoria son nulos en precio se hara un tratamiento en una puesta en comun con el equipo

In [25]:
meta_Amazon_Fashion.to_parquet('meta_Amazon_Fashion2.parquet',index=False)
fashion2.to_parquet('fashion2.parquet',index=False)

# GROCERY_AND_GOURMET_FODD

In [26]:
reviews_Grocery_and_Gourmet_Food=pd.read_parquet('reviews_Grocery_and_Gourmet_Food.parquet')
meta_Grocery_and_Gourmet_Food=pd.read_parquet('meta_Grocery_and_Gourmet_Food.parquet')

In [27]:
meta_Grocery_and_Gourmet_Food.drop(columns=['imUrl', 'related', 'salesRank'], inplace= True)
reviews_Grocery_and_Gourmet_Food.drop(columns=['helpful', 'reviewerName', 'reviewTime'], inplace=True)

In [28]:
reviews_Grocery_and_Gourmet_Food2 = convertir_minusculas(reviews_Grocery_and_Gourmet_Food)

In [29]:
meta_Grocery_and_Gourmet_Food.to_parquet('meta_Grocery_and_Gourmet_Food2.parquet',index=False)
reviews_Grocery_and_Gourmet_Food2.to_parquet('reviews_Grocery_and_Gourmet_Food2.parquet',index=False)

In [30]:
reviews_Tools_and_Home_Improvement=pd.read_parquet('reviews_Tools_and_Home_Improvement.parquet')
meta_Tools_and_Home_Improvement=pd.read_parquet('meta_Tools_and_Home_Improvement.parquet')

In [31]:
meta_Tools_and_Home_Improvement.drop(columns=['imUrl', 'related', 'salesRank'], inplace= True)
reviews_Tools_and_Home_Improvement.drop(columns=['helpful', 'reviewerName', 'reviewTime'], inplace=True)

In [32]:
reviews_Tools_and_Home_Improvement2 = convertir_minusculas(reviews_Tools_and_Home_Improvement)

In [33]:
reviews_Tools_and_Home_Improvement2.head()

,reviewerID,asin,reviewText,overall,summary,unixReviewTime
0,a15i9qv4og22a6,001212835x,this was a christmas gift for my mother for sh...,5.0,awesome lamp,1389657600
1,a1bnz90hn39tyr,0205062040,this made quick work or putting down decking. ...,5.0,awesome tool,1373846400
2,a9ducbtl8k3e4,0205062040,i had to tear off a small bedroom deck in orde...,5.0,in a word: excellent cool tool!,1382659200
3,aa8o583kryinx,0205062040,"the tool works great, as advertised.the narrow...",5.0,delivery was on time. tool works great,1372723200
4,a16cr5rcpxfmma,0205062040,"if you want to side-screw decking, this tool w...",4.0,good tool!,1364256000


In [34]:
meta_Tools_and_Home_Improvement.to_parquet('meta_Tools_and_Home_Improvement2.parquet',index=False)
reviews_Tools_and_Home_Improvement2.to_parquet('reviews_Tools_and_Home_Improvement2.parquet',index=False)

Se buscan duplicados no se encuentran,se encuentran valores nulos pero son en columnas que no seran tomadas para este analisis y precios que tiene nulos recibira un tratamiento en comun
viendo las caracterizticas de todas las categorias, se paso todo a minuscula para ser mas facil trabajarlos y validar los datos

Los archivos fueron guardados en una copia para conservar los archivos originales en caso de necesitar volver sobre los paso hechos